<a href="https://colab.research.google.com/github/elliewlh2094/Colab-pynotes/blob/main/ML2023_HW1_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業說明
- [作業說明簡報](https://speech.ee.ntu.edu.tw/~hylee/ml/ml2023-course-data/HW01.pdf)
- [【機器學習2023】 Homework 1 (introduction + sample code)
](https://www.youtube.com/watch?v=iMzxjBDMvac)
- sklearn、 TensorFlow、 xgboost 是可以使用的（使用額外線上資源請附上Reference）
- 只要 Post-processing 是由程式自動完成，且並未違反規定（如不能使用pre-trained model、不能直接
輸出hardcode 的結果、不能上網爬資料等），都是可以接受的，另外，請記得將後處理的程式一併交上，
若沒有交上，將視為違反規定。

In [ ]:
!nvidia-smi

# 下載訓練與測試資料


In [ ]:
# google drive link
# !pip install gdown
# !gdown --id '1BjXalPZxq9mybPKNjF3h5L3NcF7XKTS-' --output covid_train.csv
# !gdown --id '1B55t74Jg2E5FCsKCsUEkPKIuqaY7UIi1' --output covid_test.csv

# dropbox link
!wget -O covid_train.csv https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
!wget -O covid_test.csv https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0

# 關於資料的介紹

- 資料包含第一天調查、第一天檢測結果、第二天調查、第二天檢測結果、第三天調查、第三天檢測結果。本次作業將要嘗試預測第三天的檢測結果。


### 資料類型

- id：每一個樣本的編號。
- 州代號（State）：該樣本來自於美國哪一州，總共包含 35 個州，編碼成一個 One-hot 向量。
- 疑似 covid 的症狀（Covid-like illness）：`clu`,`ili`,...。共5項。
- 行為指標（Behavior indicators）：例如`wearing_mask`, `shop_indoors`, `restaurant_indoors`, `public_transit`, ...。共5項。
- 對於疾病防治的信念（Belief indicators）：例如 `belief_mask_effective`, `belief_distancing_effective`。共2項。
- 心理狀態指標（Mental indicators）：`worried_catch_covid`, `worried_finance`。共2項。
- 環境指標（Environment indicators）：`other_masked_public`, `other_distanced_public`,...。共3項。
- 檢測陽性案例（Tested Positive Cases）：檢測陽性，資料中的單位是百分比。共1項。

### 輸出格式

- 輸出 `.csv` 檔案，格式如下：
```
id, tested_positive
0,0.0
1,0.0
2,0.0
...
```

# 導入必要套件

### Reading/Writing Data

- `Pandas` 是Python中用來操作資料的工具，它能夠輕鬆地進行資料讀取、寫入、清洗和分析，特別適合用來處理表格資料（如CSV文件）。
- `csv` 專門用來處理CSV格式的文件，可以用來讀取和寫入CSV文件。

### For Progress Bar

- `tqdm` 是一個可以在Python中顯示進度條的工具，通常用於長時間運行的迴圈或任務中，這樣可以直觀地了解任務的進度。

### Pytorch

- TensorBoard是一個視覺化工具，可以用來顯示訓練過程中的各種指標，例如損失值、準確度等。SummaryWriter用來將這些指標記錄下來，方便後續在TensorBoard中進行可視化分析。

In [ ]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

### 定義固定的隨機數種子（seed）

- `same_seed(seed)` 這個函數的主要目的是為了確保實驗的重現性（reproducibility）。在機器學習中，很多操作都涉及隨機數生成（例如隨機初始化權重、隨機分割資料集等）。為了讓每次執行程式碼的結果一致，我們可以使用固定的隨機數種子（seed）。

    - 告訴PyTorch的後端，在使用CuDNN（NVIDIA提供的深度學習加速庫）時，使用確定性操作，以確保每次運行的結果一致。這行代碼禁用CuDNN的benchmark模式，這樣可以保證重現性，儘管有時會犧牲一些運行效率。
        ```
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        ```
    - 設置NumPy、PyTorch CPU和GPU隨機數生成器的種子。
        ```
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
        ```

### 分割成訓練集（training set）和驗證集（validation set）

- `train_valid_split(data_set, valid_ratio, seed)` 這個函數負責將給定的數據集分割成訓練集（training set）和驗證集（validation set），這在模型訓練中是非常常見的一步。

    - 根據`valid_ratio`計算出驗證集的大小。例如，如果`valid_ratio`是0.2，表示20%的數據會被用作驗證集。剩下的數據量就是訓練集的大小。
```
valid_set_size = int(valid_ratio * len(data_set))
train_set_size = len(data_set) - valid_set_size
```
    - 使用`random_split`來隨機分割數據集，同時確保分割結果的重現性。最後再將訓練集和驗證集以NumPy陣列的形式回傳，這樣可以方便後續的操作。
```
train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
return np.array(train_set), np.array(valid_set)
```

### 定義模型的推論（inference）

- `predict` 這個函數用來進行模型的推論（inference），即使用訓練好的模型來對測試集進行預測。

    - 首先將模型設置為評估模式（evaluation mode）。在這個模式下，某些特定層的行為會發生變化，例如Dropout和BatchNorm層會在評估模式下有不同的行為。初始化預測結果。
```
model.eval() # Set your model to evaluation mode.
preds = []
```
    - 開始進行推論。使用 `tqdm` 進度條來顯示預測過程，動態顯示進度。使用 `with` 上下文管理器，設定這個操作不會計算梯度，這樣可以節省記憶體和運算資源，這也是推論階段的最佳實踐。最後將結果從GPU移回CPU，並將 Tensor 從計算圖中分離出來（detach），然後加入到預測結果列表 `preds` 中。將所有的預測結果拼接成一個大 Tensor ，然後轉換成 NumPy 陣列返回。
```
for x in tqdm(test_loader):
    x = x.to(device)                        
    with torch.no_grad():                   
        pred = model(x)                     
        preds.append(pred.detach().cpu())   
preds = torch.cat(preds, dim=0).numpy()  
return preds
```

In [ ]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    return preds

# Dataset
- 這個類別的目的是將資料整齊地組織起來，使得 PyTorch 的 DataLoader 可以方便地對其進行批量操作（batching）、隨機抽樣（shuffling）等操作。

### 設定資料集類別

- 這段程式碼定義了一個名為 `COVID19Dataset` 的類別，繼承自 PyTorch 的 `Dataset` 類別，用於構建模型訓練或推論時使用的資料集。這是一個自定義的資料集類別，可以讓你靈活地控制資料的讀取方式。

    - 定義模型的輸入特徵（features）、目標值（targets）。這裡的 x 代表模型的輸入特徵，y 代表目標值，也就是模型的標籤（labels）。如果 y 是 None，這通常表示你正在進行推論（inference），而不是訓練。
    - 如果 y 不為 `None` ，則將 y 轉換為 PyTorch 的 `FloatTensor`。同樣地，將 x 轉換為 `FloatTensor`，這樣可以與 PyTorch 中的神經網路層兼容。
```
def __init__(self, x, y=None):
    if y is None:
        self.y = y
    else:
        self.y = torch.FloatTensor(y)
    self.x = torch.FloatTensor(x)
```

### 存取資料集的東西

- `__getitem__` 這個方法允許我們通過索引（index）來訪問資料集中的單個元素。在 PyTorch 的 DataLoader 中，每次迭代時都會調用這個方法來獲取一個數據點。

    - 如果 y 是 `None`，這意味著我們是在進行推論，返回對應索引的特徵 `self.x[idx]`。否則，就是在訓練或驗證時，返回對應索引的特徵 `self.x[idx]` 和目標值 `self.y[idx]`，以供模型進行學習。
```
def __getitem__(self, idx):
    if self.y is None:
        return self.x[idx]
    else:
        return self.x[idx], self.y[idx]
```

### 取得資料集大小

- `__len__` 這個方法返回資料集的長度，即資料集中有多少個數據點。這對於迭代整個資料集來說是必須的，DataLoader 會根據這個長度來確定總共要迭代多少次。



In [ ]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# 定義 Neural Network Model
- **挑戰：Try out different model architectures by modifying the class below.**
- 這段程式碼定義了一個名為 `My_Model` 的神經網路模型。這個模型是繼承自 PyTorch 的 `nn.Module` 類別，用於處理回歸任務（如預測某些連續值）。模型的架構相對簡單，但你可以根據需要進行調整和改進。

### 定義模型的結構

- 先呼叫父類的初始化方法，確保 `nn.Module` 的一些內部機制得以正確初始化。使用了 PyTorch 的 `nn.Sequential` 來構建模型的層次結構。

    - 第一層是全連接層（線性層），將輸入的維度從 input_dim 映射到16維。接著是一層激勵函數 ReLU，它會將輸入中小於0的值置為0，這樣可以引入非線性，提高模型的表現能力。
    - 第二層也是全連接層，將16維輸出映射到8維。再一次使用 ReLU 激勵函數。
    - 第三層仍然是全連接層，將8維輸出映射到1維，這通常是回歸任務的最後一層，用來預測單一值。
```
def __init__(self, input_dim):
    super(My_Model, self).__init__()
    self.layers = nn.Sequential(
        nn.Linear(input_dim, 16),
        nn.ReLU(),
        nn.Linear(16, 8),
        nn.ReLU(),
        nn.Linear(8, 1)
    )
```

### 定義了模型的前向傳播（forward pass）

- `forward` 這個方法定義了模型的前向傳播（forward pass）過程，當你將輸入數據傳入模型時，這個方法會被自動調用，計算並返回模型的輸出。

    - 輸入數據（特徵） `x` 依次通過 `__init__` 中定義的網路層結構（`nn.Sequential` 的部分）進行計算。
    - 因為最終輸出是來自於最後一個線性層 `nn.Linear(8, 1)` 會生成一個形狀為 (B, 1) 的 Tensor，即：每個批次中的每個樣本對應一個預測值。使用 x.squeeze(1) 後，這個形狀變為 (B)，每個批次的樣本直接對應一個純量值，這樣更符合常見的損失函數要求，並且使後續處理更為簡單和直觀。
```
def forward(self, x):
x = self.layers(x)
x = x.squeeze(1) # (B, 1) -> (B)
return x
```


In [ ]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# 進行特徵選擇（Feature Selection）

- **挑戰：Try better feature selection.**
- 特徵選擇是在機器學習中一個重要的步驟，它可以幫助你挑選出對模型訓練最有幫助的特徵，從而提高模型的效能並減少過擬合的風險。
- 這個函式還有一個參數 `select_all`，用來控制是否選擇所有特徵。如果設定為 `True`，則選擇所有特徵；否則，你可以指定要選擇的特徵。

### 資料分割

- 將資料分割為特徵（x）和目標值（y）。

    - `train_data[:,-1]` 和 `valid_data[:,-1]` 代表訓練數據和驗證數據的最後一行，通常這一行是目標值 y，這裡將其分配給 `y_train`。（在資料集中是代表`tested_positive`。）
    - `train_data[:,:-1]` 和 `valid_data[:,:-1]` 代表訓練數據和驗證數據所有行（除了最後一行），這些是模型的輸入特徵 x，這裡將其分配給 `raw_x_train` 和 `raw_x_valid`。
    - `test_data` 直接分配給 `raw_x_test`，假設測試數據不包含目標值 y，所以只取特徵列。
```
y_train, y_valid = train_data[:,-1], valid_data[:,-1]
raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data
```


### 特徵選擇邏輯

- 這裡是決定是否選擇所有資料。若否，則從原始資料中，根據資料特性或實驗需求修改這個範圍。

    - 最後，函數返回選擇出的特徵以及相應的目標變量。返回的值包括：
```
if select_all:
    feat_idx = list(range(raw_x_train.shape[1]))
else:
    feat_idx = list(range(35, raw_x_train.shape[1])) # TODO: Select suitable feature columns.
return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid
```

In [ ]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1])) # Select all.
    else:
        feat_idx = list(range(35, raw_x_train.shape[1])) # TODO: Select suitable feature columns.
        # Exclude the indices
        # exclude_idx = [46, 51, 64, 69, 82, 87]
        # feat_idx = [i for i in range(35, raw_x_train.shape[1]) if i not in exclude_idx]

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# 定義訓練方式

- **挑戰： try more Different optimizers and L2 regularization.**
- 這是模型訓練過程的核心部分，它負責優化模型的參數以減少損失函數的值。

### 初始化設定

- 定義損失函數（Loss Function）：這裡使用了均方誤差損失函數（MSELoss），這種損失函數常用於回歸問題，衡量預測值與真實值之間的差異。reduction='mean' 表示最終的損失是所有樣本損失的平均值。
- 定義最佳化演算法（Optimization Algorithm）：這裡選用了隨機梯度下降（SGD）作為優化演算法，並設定了學習率（learning rate）和動量（momentum）。動量可以加速收斂，避免訓練過程中的震盪。你可以嘗試不同的優化器來比較效果。
- TensorBoard 可視化工具：SummaryWriter 用於將訓練過程中的數據（如損失值）記錄下來，以便後續在 TensorBoard 中進行可視化分析。
- 模型儲存目錄：檢查並創建一個目錄，用來保存訓練過程中表現最好的模型。
- 設定參數：初始化訓練的回合數（n_epochs）、最佳損失（best_loss）、步驟計數器（step）、以及早停計數器（early_stop_count）。
```
    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7)

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.
    
    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0
```

### 訓練設定
- 訓練模式：每個訓練回合都將模型設定為訓練模式（train mode），這樣會啟用 `dropout` 等層，計算梯度以更新模型參數。
- 進度條：使用 `tqdm` 來顯示訓練進度條（Progess Bar），這樣可以直觀地了解訓練進度。
- 訓練迴圈步驟
    1. 將梯度歸零，避免累加上一次的梯度。
    2. 將資料移動到指定的運算設備（例如 GPU）。
    3. 將輸入資料傳入模型進行預測。
    4. 計算預測值與目標值之間的損失。
    5. 反向傳播，計算梯度。
    6. 更新模型參數。
    7. 記錄當前批次的損失值。
    8. 顯示進度條，包含訓練週期與當前損失。

-
```
for epoch in range(n_epochs):
    model.train() # Set your model to train mode.
    loss_record = []

    # tqdm is a package to visualize your training progress.
    train_pbar = tqdm(train_loader, position=0, leave=True)

    for x, y in train_pbar:
        optimizer.zero_grad()               # Set gradient to zero.
        x, y = x.to(device), y.to(device)   # Move your data to device.
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()                     # Compute gradient(backpropagation).
        optimizer.step()                    # Update parameters.
        step += 1
        loss_record.append(loss.detach().item())

        # Display current epoch number and loss on tqdm progress bar.
        train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
        train_pbar.set_postfix({'loss': loss.detach().item()})
```


### 記錄訓練的損失
- 計算訓練集的平均損失，並使用 TensorBoard 記錄下來。
```
    mean_train_loss = sum(loss_record)/len(loss_record)
    writer.add_scalar('Loss/train', mean_train_loss, step)
```

### 驗證迴圈設定
- 驗證模式：將模型設置為驗證模式（eval mode），這樣會禁用 `dropout` 等層。然後對驗證集進行推論並計算損失。
- 驗證迴圈步驟
    1. 將資料移動到指定的運算設備（例如 GPU）。
    2. 禁用梯度計算，因為在驗證過程中不需要更新模型參數。
    3. 將輸入資料傳入模型進行預測。
    4. 計算預測值與目標值之間的損失。
    5. 記錄當前批次的損失值。
-
```
        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
```

### 記錄驗證的損失
- 計算驗證集的平均損失，並使用 TensorBoard 記錄，同時在控制台輸出。
```
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)
```

### 儲存模型與早停策略
- 儲存最佳模型：如果當前回合的驗證損失低於之前的最佳損失，則更新最佳損失並保存當前模型。這樣可以確保最終保存的模型是在訓練過程中表現最好的版本。

- 早停策略：如果驗證損失在連續多個回合（由 `early_stop` 設定）沒有改善，則停止訓練，這樣可以避免模型過度訓練（overfitting）
```
        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return
```

In [ ]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm.
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7)
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device.
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        # 儲存模型與早停策略
        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

        # Show final result in the end of training
        if epoch+1 == n_epochs:
            print(f'Best loss: {best_loss:.4f}')

# Configurations
- **挑戰： Try more hyperparameters.**
- `config` contains hyper-parameters for training and the path to save your model.
- 設備（Device）：這行代碼決定了模型訓練運行在哪個設備上。如果你的電腦或環境（例如Colab）支持CUDA（NVIDIA的GPU運算框架），那麼設備將設置為 'cuda'，這樣會利用GPU加速運算。否則，將使用CPU進行運算。
- `seed`: 設定隨機數種子（seed），這樣可以確保每次運行結果一致。這裡隨意選了一個數字，你可以更改為任何你喜歡的數字。seed 的範圍通常是 0 到 2^{32}-1 （即 0 到 4,294,967,295）。超出這個範圍的數值可能會導致錯誤或自動被截斷到這個範圍內。

- `select_all`: 設定是否使用所有的特徵。如果設置為 `True`，則模型會使用所有可用的特徵；如果設置為 False，則只會使用部分特徵。

- `valid_ratio`: 設定訓練集和驗證集的比例。這裡設定為`0.2`，表示驗證集的大小是訓練集的20%。

- `n_epochs`: 訓練的回合數（epoch）的總數。每個回合代表模型在整個訓練集上運行一次，這裡設定為`5000`，表示模型將在訓練集上進行5000次完整的訓練。

- `batch_size`: 批次大小（batch size），即每次訓練時從訓練集中取出多少樣本來進行運算。較大的批次大小可以加速運算，但也需要更多的記憶體資源。這裡設定為256。

- `learning_rate`: 學習率（learning rate），控制每次更新模型參數時步伐的大小。這是訓練中的一個關鍵超參數，設定過大會導致模型無法收斂，設定過小會使訓練過程非常緩慢。這裡設定為1e-5，是一個較小的學習率。

- `early_stop`: 早停策略的設置，當模型連續 `early_stop` 次回合沒有改善時，停止訓練。這裡設定為600，表示如果模型在600個回合內驗證集損失沒有改善，則停止訓練以防止過擬合。

- `save_path`: 模型儲存路徑，訓練過程中表現最好的模型會被保存到這個路徑中。這裡設定為 `./models/model.ckpt`，你可以根據需要更改這個路徑。



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 8881234,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.
    'batch_size': 256,
    'learning_rate': 1e-5,
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

# Dataloader

- 進行資料的前處理與資料加載，並將資料集轉換為 PyTorch 的 DataLoader 格式，以便在後續的模型訓練和推論中使用。

- 設定隨機數種子：這行程式碼呼叫之前定義的 same_seed 函數，設置隨機數種子來確保結果的重現性。
- 讀取資料：從 CSV 文件中讀取訓練資料和測試資料，並將它們轉換為 NumPy 陣列。
    - `train_data` 的大小是 3009 x 89，包含 1 個 ID、34 個州代碼、18 個特徵，以及這些特徵在過去三天的數據。
    - `test_data` 的大小是 997 x 88，因為測試資料不包括最後一天的陽性率。
- 分割資料：使用最前面設定的 `train_valid_split` 函數將訓練資料進一步分割為訓練集和驗證集。這裡使用了 `config['valid_ratio']` 來控制驗證集的比例。
- 選擇特徵：呼叫 `select_feat` 函數從資料集中選擇出要用於模型訓練的特徵。根據 `config['select_all']` 的設置，可能會選擇全部或部分特徵。
- 建立資料集：這裡使用 `COVID19Dataset` 類別創建訓練集、驗證集和測試集的資料集物件。x_train、x_valid 和 x_test 是特徵，y_train 和 y_valid 是目標變數（標籤）。對於測試集，沒有 y_test，因為在推論階段我們不需要這些標籤。
- 資料載入器（DataLoader）：DataLoader 是 PyTorch 中用來迭代資料集的工具。它將資料集分批（batch）加載到模型中，並根據需要隨機打亂資料（`shuffle=True`）以提高模型的通用能力。
    - `train_loader`：用於訓練資料，批次大小由 `config['batch_size']` 控制，並啟用隨機打亂。
    - `valid_loader`：用於驗證資料，也設置了批次大小並啟用了隨機打亂。
    - `test_loader`：用於測試資料，批次大小設置為相同，但不進行隨機打亂（`shuffle=False`），因為測試過程不需要隨機化。

In [ ]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 3009 x 89 (1 ID + 34 states + 18 features x 3 days)
# test_data size: 997 x 88 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

# 訓練模型

- 初始化模型：這行程式碼創建了一個 `My_Model` 模型實例，`input_dim=x_train.shape[1]` 表示模型的輸入維度根據訓練資料的特徵數量來設定。`to(device)` 將模型移動到指定的運算設備（CPU或GPU）上，這樣可以加速運算。
- 訓練模型：`trainer` 函數隨後被呼叫，這是你之前定義的訓練函數，它負責使用訓練資料和驗證資料對模型進行訓練。

In [ ]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

# Plot learning curves with `tensorboard`

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# 保存模型

- 保存預測結果：`save_pred` 這段函數用於將模型的預測結果保存到一個 CSV 文件中。它會將每個樣本的編號（id）與預測的陽性率（tested_positive）寫入文件。
- 載入最佳模型：這裡重新初始化了一個 My_Model 模型，並載入了之前訓練過程中保存的最佳模型參數（使用 `config['save_path']` 路徑）。
- 進行預測：呼叫 `predict` 函數，使用測試資料集對模型進行推論，得到預測結果 preds。
- 保存預測結果：使用 `save_pred` 函數將預測結果保存到 pred.csv 文件中。

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')

- 下載測試結果：這段程式碼使用 IPython.display 模組生成一個 pred.csv 文件的下載鏈接，方便你在 Jupyter Notebook 或 Colab 環境中直接下載預測結果。

In [ ]:
from IPython.display import FileLink
FileLink(r'pred.csv')